## Module 1 Project: Analysis of Treasury Bill Pricing and Risk

### Introduction
Treasury bills, or T-bills, are debt instruments with short-term maturity periods ranging from a few days to 52 weeks. T-bills are zero-coupon fixed-income investments, i.e., no coupon payments are paid during their term. Instead, treasury bills are priced so that the bill holder receives the par value $V_{P}$ at the end of the term.

A zero-coupon T-bill has an annual market interest rate of $\bar{r}$, specified at the time of purchase, and a T-year term. The fair price of the treasury bill is the future face (par) value discounted to today by the market interest rate $\bar{r}$:

$$
V_{B} = \frac{V_{P}}{\left(1+\bar{r}\right)^{T}}
$$

The quantity $V_{B}$ denotes the current price, and $V_{P}$ represents the par (face) value of the T-bill.  Treasury bills are issued for specific periods, such as 4, 8, 13, 26, and 52 weeks.

### Scenario
You purchased a 52-week T-bill with a par value $V_{P}=1000$ USD and interest rate $\bar{r}$. However, four weeks (28-days) after purchasing the T-bill, the interest rate is now $\bar{r}^{\prime}$. Should you `sell` the T-bill before it matures or `hold` it to term?

#### Tasks
 1. Load the daily historical interest rate data from 2019 to 2022.
 1. Develop a backetesting strategy using historical interest rate data to answer the sell versus hold question.

### Setup
The `Include.jl` file loads [Julia functions](https://docs.julialang.org/en/v1/manual/functions/) and [problem specific types](https://docs.julialang.org/en/v1/manual/functions/) that will be helpful for the analysis of T-bill pricing. In addition, `Include.jl` loads several external packages that we will use for our excercise:
* [DataFrames.jl](https://dataframes.juliadata.org/stable/) provides a set of tools for working with tabular data in [Julia](https://julialang.org). Its design and functionality are similar to those of [Pandas (in Python)](https://pandas.pydata.org) and [data.frame, data.table and dplyr (in R)](https://dplyr.tidyverse.org), making it a great general purpose data science tool.

In [1]:
include("Include.jl");

  Activating project at `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-131/module-1`


#### Functions
* The `loadratesfile(year::String = "2022") -> DataFrame` function takes a [String](https://docs.julialang.org/en/v1/manual/strings/) encoding the year in the `YYYY` format, e.g., `"2019"` or `"2020"` and returns a [DataFrame](https://dataframes.juliadata.org/stable/) holding the daily interest rates for US Treasury Bills for various durations found on [Treasury.gov](https://home.treasury.gov/resource-center/data-chart-center/interest-rates/TextView?type=daily_treasury_bill_rates). This project has data from `"2019"`, `"2020"`, `"2021"`, `"2022"` and `"2022"`. The year parameter has a default value of `"2022"`.
* The `build(model::Type{MyUSTreasuryBillModel}, data::NamedTuple) -> MyUSTreasuryBillModel` function takes information in the `data` arguement (the par value, the rate and the duration of the T-bill) and returns an instance of the `MyUSTreasuryBillModel` custom type.

#### Types
* `MyUSTreasuryBillModel` is a [mutable type](https://docs.julialang.org/en/v1/manual/types/#Mutable-Composite-Types) holding the par value $V_{P}$, the duration $T$ and the interest rate $\bar{r}$ for a zero-coupon treasury bill. You construct a `MyUSTreasuryBillModel` instance using the `build` method described above.

### Task 1: Load historical interest rate data
 The rates are based on recently auctioned bills with various maturities and collected by the Federal Reserve Bank of New York every business day. The Bank Discount rate is based on par value and discount amount, while the Coupon Equivalent is based on purchase price and discount, and can be used to compare yields.

In [2]:
rates = loadratesfile(year="2022")

Row,Date,4 WEEKS BANK DISCOUNT,4 WEEKS COUPON EQUIVALENT,8 WEEKS BANK DISCOUNT,8 WEEKS COUPON EQUIVALENT,13 WEEKS BANK DISCOUNT,13 WEEKS COUPON EQUIVALENT,17 WEEKS BANK DISCOUNT,17 WEEKS COUPON EQUIVALENT,26 WEEKS BANK DISCOUNT,26 WEEKS COUPON EQUIVALENT,52 WEEKS BANK DISCOUNT,52 WEEKS COUPON EQUIVALENT
,String15,Float64,Float64,Float64,Float64,Float64,Float64,Float64?,Float64?,Float64,Float64,Float64,Float64
1,12/30/2022,3.95,4.02,4.3,4.39,4.3,4.4,4.57,4.7,4.6,4.77,4.51,4.73
2,12/29/2022,3.88,3.95,4.28,4.37,4.34,4.45,4.54,4.67,4.57,4.74,4.48,4.7
3,12/28/2022,3.57,3.63,4.08,4.16,4.35,4.46,4.55,4.68,4.59,4.76,4.49,4.71
4,12/27/2022,3.62,3.68,4.09,4.17,4.35,4.46,4.49,4.62,4.6,4.77,4.52,4.75
5,12/23/2022,3.65,3.71,4.01,4.09,4.23,4.33,4.45,4.58,4.52,4.69,4.46,4.67
6,12/22/2022,3.65,3.71,4.02,4.1,4.24,4.34,4.43,4.56,4.51,4.68,4.45,4.66
7,12/21/2022,3.66,3.72,4.01,4.09,4.22,4.32,4.44,4.57,4.51,4.68,4.41,4.62
8,12/20/2022,3.69,3.75,4.02,4.1,4.25,4.36,4.4,4.53,4.55,4.72,4.44,4.65
9,12/19/2022,3.77,3.83,4.03,4.11,4.28,4.39,4.41,4.54,4.55,4.72,4.44,4.65
